In [1]:
%matplotlib inline

import numpy as np
import numpy.random as npr
import math
import matplotlib.pyplot as plt

from tqdm import tqdm

from rlbase import env_bandit, policy, misc, agent_simple

ModuleNotFoundError: No module named 'misc'

In [ ]:
#import sys
#sys.path.append("/home/hofmann/nngame/package/src/rlbase/")

#import misc
#import env_bandit
#import policy
#import agent_simple
#import agent_gradient

In [2]:
def plot_results(results, optimal_actions):
    plt.figure(figsize=(15, 5), dpi= 80, facecolor='w', edgecolor='k')
    plt.subplot(2,1,1)
    for k in results.keys():
      plt.plot(results[k])
    plt.legend(results.keys())
    plt.subplot(2,1,2)
    for k in optimal_actions.keys():
      plt.plot(optimal_actions[k])
    plt.legend(optimal_actions.keys())
    plt.show()

# Figure 2.2
Average performance of $\epsilon$-greedy action-value methods on the 10-armed testbed. These data are averages over 2000 runs with different bandit problems. All methods used sample averages as their action-value estimates.

In [ ]:
N_timesteps = 1000
N_bandits = 2000

agent_infos = {"$\epsilon$=0":{"eps":0},
               "$\epsilon$=0.01":{"eps":0.01},
               "$\epsilon$=0.1":{"eps":0.1}}

results         = {i:[0]*N_timesteps for i in agent_infos}
optimal_actions = {i:[0]*N_timesteps for i in agent_infos}
for bandit in tqdm(range(N_bandits)):
  env = rlbase.env_bandit.BanditEnvironment()
  for (k,v) in agent_infos.items():
    pi = policy.EpsGreedy(env=env, **v)
    agent = agent_simple.SimpleAgent(env=env, pi=pi)  
    
    best_action = list(env.arms).index(max(env.arms))
    s = env.start()
    a = agent.start(s)
    for t in range(N_timesteps):
      r, s, _ = env.step(s,a)
      a = agent.step(r,s)
      results[k][t] += r / N_bandits
      optimal_actions[k][t] += (a==best_action) / N_bandits
plot_results(results, optimal_actions)

# Exercise 2.5
Demonstrate the difficulties that sample-average methods have for nonstationary problems.

In [ ]:
N_timesteps = 10000
N_bandits = 1000

agent_infos = {"SampleAverage":{"eps":0.1},
               "ConstantStepsize":{"eps":0.1,"alpha":0.1}}

results         = {i:[0]*N_timesteps for i in agent_infos}
optimal_actions = {i:[0]*N_timesteps for i in agent_infos}
for bandit in tqdm(range(N_bandits)):
  env = env_bandit.BanditEnvironment(random=True)
  for (k,v) in agent_infos.items():
    pi = policy.EpsGreedy(env=env, **v)
    agent = agent_simple.SimpleAgent(env=env,pi=pi,**v)
    s = env.start()
    a = agent.start(s)
    for t in range(N_timesteps):
      best_action = list(env.arms).index(max(env.arms))
      r, s, _ = env.step(s,a)
      a = agent.step(r,s)
      results[k][t] += r / N_bandits
      optimal_actions[k][t] += (a==best_action) / N_bandits
plot_results(results, optimal_actions)

# Figure 2.3
The effect of optimistic initial action-value estimates on the 10-armed testbed. Both methods use a constant step-size parameter, $\alpha=0.1$.

In [ ]:
N_timesteps = 1000
N_bandits = 1000

agent_infos = {"$\epsilon$=0.1 Q=0":{"eps":0.1},
               "$\epsilon$=0  Q=5":{"eps":0,"q_init":5}}

results         = {i:[0]*N_timesteps for i in agent_infos}
optimal_actions = {i:[0]*N_timesteps for i in agent_infos}
for bandit in tqdm(range(N_bandits)):
  env = env_bandit.BanditEnvironment()
  for (k,v) in agent_infos.items():
    pi = policy.EpsGreedy(env=env,eps=0.1)
    agent = agent_simple.SimpleAgent(env=env,pi=pi,alpha=0.1,**v)
    s = env.start()
    a = agent.start(s)
    best_action = list(env.arms).index(max(env.arms))
    for t in range(N_timesteps):
      r, s, _ = env.step(s,a)
      a = agent.step(r,s)
      results[k][t] += r / N_bandits
      optimal_actions[k][t] += (a==best_action) / N_bandits
plot_results(results, optimal_actions)

# Figure 2.4
Average performance of UCB action selection on the 10-armed testbed. As shown, UCB generally performs better than $\epsilon$-greedy action selection, except in the first $k$ steps, when it selects randomly among the as-yet-untried actions.

In [ ]:
N_timesteps = 1000
N_bandits = 1000

agent_infos = {"$\epsilon$=0.1":{"eps":0.1},
               "UCB c=2":{"eps":0,"ucb_c":2}}

results         = {i:[0]*N_timesteps for i in agent_infos}
optimal_actions = {i:[0]*N_timesteps for i in agent_infos}
for bandit in tqdm(range(N_bandits)):
  env = env_bandit.BanditEnvironment()
  for (k,v) in agent_infos.items():
    pi = policy.EpsGreedy(env=env,**v)
    agent = agent_simple.SimpleAgent(env=env,pi=pi,**v)
    s = env.start()
    a = agent.start(s)
    best_action = list(env.arms).index(max(env.arms))
    for t in range(N_timesteps):
      r, s, _ = env.step(s,a)
      a = agent.step(r,s)
      results[k][t] += r / N_bandits
      optimal_actions[k][t] += (a==best_action) / N_bandits
plot_results(results, optimal_actions)

# Figure 2.5
Average performance of the gradient bandit algorithm with and without a reward baseline on the 10-armed testbed when the $q_*(a)$ are chosen to be near $+4$ rather than near zero.

In [ ]:
N_timesteps = 1000
N_bandits = 1000

agent_infos = {r"Without baseline,$\alpha$=0.1":{"baseline":False,"alpha":0.1},
               r"With baseline,$\alpha$=0.1":{"baseline":True,"alpha":0.1},
               r"Without baseline,$\alpha$=0.4":{"baseline":False,"alpha":0.4},
               r"With baseline,$\alpha$=0.4":{"baseline":True,"alpha":0.4}}

results         = {i:[0]*N_timesteps for i in agent_infos}
optimal_actions = {i:[0]*N_timesteps for i in agent_infos}
for bandit in tqdm(range(N_bandits)):
  env = env_bandit.BanditEnvironment(offset=4)
  for (k,v) in agent_infos.items():
    pi = policy.Softmax(env=env)
    agent = agent_gradient.GradientAgent(env=env,pi=pi,**v)
    s = env.start()
    a = agent.start(s)
    best_action = list(env.arms).index(max(env.arms))
    for t in range(N_timesteps):
      r, s, _ = env.step(s,a)
      a = agent.step(r,s)
      results[k][t] += r / N_bandits
      optimal_actions[k][t] += (a==best_action) / N_bandits
plot_results(results, optimal_actions)

# Figure 2.6
A parameter study of the various bandit algorithms presented in this chapter. Each point is the average reward obtained over 1000 steps with a particular algorithm at a particular setting of its parameter

In [ ]:
N_timesteps = 1000
N_bandits = 1000

agent_infos = {}
for eps in np.power(2.,np.arange(-7,-1,.5)):
    key = ("$\epsilon$-greedy",eps)
    value = {"eps": eps}
    agent_infos[key] = value
for ucb_c in np.power(2.,np.arange(-4,3,.5)):
    key = ("UCB",ucb_c)
    value = {"eps": 0, "ucb_c": ucb_c}
    agent_infos[key] = value
for val in np.power(2.,np.arange(-2,3,.5)):
    key = ("Optimistic",val)
    value = {"eps": 0, "q_init": val,"alpha":0.1}
    agent_infos[key] = value
for alpha in np.power(2.,np.arange(-5,3,.5)):
    key = ("Gradient",alpha)
    value = {"alpha": alpha}
    agent_infos[key] = value
     
results         = {i:0 for i in agent_infos.keys()}
optimal_actions = {i:0 for i in agent_infos.keys()}
for bandit in tqdm(range(N_bandits)):
  env = env_bandit.BanditEnvironment()
  for (k,v) in agent_infos.items():
    if k[0]=="Gradient":
        pi = policy.Softmax(env=env)
        agent = agent_gradient.GradientAgent(env=env,pi=pi,**v)
    else:
        pi = policy.EpsGreedy(env=env,**v)
        agent = agent_simple.SimpleAgent(env=env,pi=pi,**v)       
    s = env.start()
    a = agent.start(s)
    best_action = list(env.arms).index(max(env.arms))
    for t in range(N_timesteps):
      r, s, _ = env.step(s,a)
      a = agent.step(r,s)
      results[k] += r
      optimal_actions[k] += (a==best_action)

In [ ]:
# Import the necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

values_algo = [a for (a,b) in results.keys()]
values_x    = [b for (a,b) in results.keys()]
values_y    = [v/bandit/N_timesteps for v in results.values()]
df = pd.DataFrame({"x":values_x,"y":values_y,"algo":values_algo})

cols = {"$\epsilon$-greedy":"r","UCB":"b","Optimistic":"k","Gradient":"g"}
plt.figure(figsize=(15, 5), dpi= 80, facecolor='w', edgecolor='k')
for a in list(set(df.algo)):
    d = df[df["algo"]==a]
    plt.semilogx(d.x,d.y,cols[a])
plt.legend(list(set(df.algo)))

exps = np.arange(-7,3)
xticks = np.power(2.,exps)
labels = [r"$2^{"+str(e)+"}$" for e in exps]

plt.xticks(xticks,labels)
plt.ylim(1,1.5)
plt.show()